## **Лабороторная работа 3 "Кластерный анализ методом DBSCAN"**

####**Экономика городов**

Данные описывают экономические условия в 48 городах мира в 1991 году.
Данные были собраны отделом экономических исследований банка Union (Швейцария).

Описаны экономические условия в 48 городах мира в 1991 году.

**Число наблюдений**: 48

**Переменные:**
| Название переменных | Описание |
|------------------|-----------|
| **City (Город)** | Название города |
| **Работа (Work)** | Взвешенное среднее числа рабочих часов, сосчитанное по 12 профессиям |
| **Цена (Price)** | Индекс цен 112 товаров и услуг, включая арендную плату за жилье (значение для Цюриха взято за 100%) |
| **Заработная плата (Salary)** | Индекс заработной платы за час работы, сосчитанный по 12 профессиям после налогов и вычетов (значение для Цюриха взято за 100%) |

#### **Подготовка данных**

In [ ]:
#Загружаем данные
!gdown 1CgelswAHDrIJA-evy5wF4zjIdBFNuq75

Downloading...
From: https://drive.google.com/uc?id=1CgelswAHDrIJA-evy5wF4zjIdBFNuq75
To: /content/Econom_Cities_data.csv
100% 1.18k/1.18k [00:00<00:00, 1.70MB/s]


In [ ]:
# Импортируем необходимые для работы библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [ ]:
# Импортируем данные
df = pd.read_csv('Econom_Cities_data.csv', sep=';', decimal=',', index_col='City')
df.head(10)

,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5
Buenos_Aires,1971,56.1,12.5
Cairo,-9999,37.1,-9999.0
Caracas,2041,61.0,10.9
Chicago,1924,73.9,61.9


In [ ]:
# Смотрим описательную статистику
df.describe()

,Work,Price,Salary
count,48.000000,48.000000,48.000000
mean,1384.958333,68.860417,-378.727083
std,2404.897007,21.784659,2027.338052
min,-9999.000000,30.300000,-9999.000000
25%,1740.750000,49.650000,12.250000
50%,1834.500000,70.500000,40.300000
75%,1972.750000,81.700000,58.700000
max,2375.000000,115.500000,100.000000


Замечаем, что значения min по столбцам 'Work' и 'Salary' составляют -9999, что не может быть правдой. Скорее всего, так обозначили пропущенные данные (NaN). Уберем эти значения из выборки.

In [ ]:
df = df.replace(-9999, np.nan).dropna(subset=['Work', 'Salary'])
df.describe()

,Work,Price,Salary
count,46.000000,46.000000,46.000000
mean,1879.913043,70.100000,39.545652
std,174.342552,21.389177,24.757703
min,1583.000000,30.300000,2.700000
25%,1745.250000,51.775000,14.375000
50%,1849.000000,70.950000,43.650000
75%,1976.250000,81.900000,59.700000
max,2375.000000,115.500000,100.000000


Следующим шагом необходимо стандартизировать данные, так как они имеют разные диапазоны значений и измеряются в разных величинах. Воспользуемся формулой StandardScaler (приведение к нулевому среднему и единичной дисперсии)

**Формула**
$$
\mathrm{StandardScaler} \;=\; \frac{\mathrm{x} - \mathrm{mean}}{\mathrm{std}}
$$

In [ ]:
# Стандартизируем
X = df
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#### **Кластерный анализ методом DBSCAN**

In [ ]:
# Создадим объект DBSCAN при eps = 2
dbscan = DBSCAN(eps=2, metric='euclidean', min_samples=2)
# Обучаем модель DBSCAN
dbscan.fit(X_scaled)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ 0 46]]


Все объекты оказались в одном кластере. Такой вариант не подходит. Необходимо уменьшить eps.

In [ ]:
# Создадим объект DBSCAN при eps = 1
dbscan = DBSCAN(eps=1, metric='euclidean', min_samples=2)
# Обучаем модель DBSCAN
dbscan.fit(X_scaled)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  4]
 [ 0 38]
 [ 1  2]
 [ 2  2]]


Такой вариант тоже не подходит. В 0 кластере слишком много объектов по сравненеию с другими. Пробуем дальше. Уменшим eps до 0.8 и увеличим min_samples до 3.

In [ ]:
# Создадим объект DBSCAN при eps = 0.8
dbscan = DBSCAN(eps=0.8, metric='euclidean', min_samples=3)
# Обучаем модель DBSCAN
dbscan.fit(X_scaled)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


Результат уже лучше. Однако получилось много выбросов (~17%). Попробуем сделать еще итерацию.

In [ ]:
# Создадим объект DBSCAN при eps = 0.75 и min_samples=2
dbscan = DBSCAN(eps= 0.75, metric='euclidean', min_samples=2)
# Обучаем модель DBSCAN
dbscan.fit(X_scaled)

unique, counts = np.unique(dbscan.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  4]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  2]]


Данный результат кажется более приемлемым. Дальнейшее уменьшение eps приводит к увеличению выбросов и излишнему разбиению на кластеры, что в дальнейшем будет сложно интрепретировать. Увеличение min_samples также не кажется рациональным, так как это приводит к увеличению числа выбросов.

Таким образом, мы **получаем 4 кластера и 4 выброса (шум).**

In [ ]:
df['dbscan'] = dbscan.labels_
df['dbscan'].sort_values()

,dbscan
City,
Hong_Kong,-1
Tokyo,-1
Taipei,-1
Stockholm,-1
Dublin,0
Amsterdam,0
Chicago,0
Brussels,0
Houston,0


In [ ]:
df.groupby('dbscan').mean()

,Work,Price,Salary
dbscan,,,
-1,2051.250000,93.600000,42.375000
0,1792.000000,77.526316,55.157895
1,1959.210526,50.115789,14.789474
2,1874.000000,97.950000,95.150000
3,1625.000000,114.550000,65.150000


#### **Выводы**
На основе проведенного кластерного анализа методом DBSCAN было выделено 4 кластера и 4 выброса (шум).

**-1 (шум)** - города, где сочетается очень высокая продолжительность рабочего дня, повышенные цены и заработная плата немного ниже среднего уровня. Скорее всего, жители данных городов вынуждены много работать для поддержания нормального уровня жизни, но при этом не могут достичь высокого уровня благосостояния.

**0 кластер** - города, где наблюдается наиболее сбалансированная экономическая модель, все показатели находятся на "среднем" уровне.

**1 кластер** - города с низким уровне жизни, наблюдается низкий уровень заработных плат (при относительно высоком уровне цен) и одновременно высокая продолжительность рабочего времени.

**2 кластер** - города с высоким уровнем жизни, наблюдается высокие значения индекса цен и индекса заработной платы, при этом сохраняя умеренную продолжительность рабочего дня.

**3 кластер** - города с самой короткой продолжительностью рабочего дня и самым высоким индексом цен при среднем уровне заработных плат. Скорее всего, жители данных городов отдают предпочтение не карьерному росту, а свободному времени. То есть высокий уровень цен как бы компенсируется за счет увеличения личного времени.

### **Общий вывод по результатам Лабороторных работ №1-3**

В результате выполнения Лабороторных работ №1-3, были рассмотрены три подхода для кластеризации:
*   Иерархический кластерный анализ (построение дендрограммы);
*   Кластерный анализ методом к-средних (построение графика "Каменистая осыпь");
*   Кластерный анализ методом DBSCAN (области в пространстве с высокой плотностью точек).

В работе был использован датасет "Econom_Cities_data.csv", описывающий экономические условия в 48 городах мира в 1991 году.

**Полученные результаты:**

| Подход | Количество кластеров | Выбросы |
|------------|-----------|-----------|
| **Иерархический кластерный анализ** | 5 | - |
| **Кластерный анализ методом к-средних** | 5 | - |
| **Кластерный анализ методом DBSCAN** | 4 | 4 |

Каждый метод кластеризации имеет свои достоинства и недостатки.

Иерархический кластерный анализ представляет собой достаточно простой в понимании метод, который позволяет визуально представить кластеры в формате дендрограммы, важным преимуществом является то, что не требуется заранее задавать число кластеров. Однако, данный метод может быть сложен в интерпретации, особенно для новичков, так как можно ошибочно определить слишком большое число кластеров (или наоборот маленькое), что приведет к некорректной интерпретации результатов.

Кластерный анализ методом к-средних является также достаточно простым методом для понимания и реализации. При построении графика "Каменистая осыпь" можно наглядно увидеть "точку преломления", где график начинает становиться более пологим, а значит качество кластеризации ухудшается. Тем не менее, использования данного метода предполгает субъективное решение со стороны аналитика, так как он задает число кластеров.

Кластерный анализ методом DBSCAN объявляет кластерами области в пространстве с высокой плотностью точек, при этом отбрасывая "шумовые" точки (выбросы, которые не попадают в заданный радиус). Достоинством данного метода является и то, что он сам определяет число кластеров и может находить кластеры достаточно сложной формы. Однако трудность и субъективизм данного метода связан с подбором оптимальных значений eps (радиус) и min_samples (минимальное кол-во точек, входящих в окрестность).

В рамках данной работы, я считаю, что наиболее удачным является Кластерный анализ методом к-средних. На мой взгляд, он позволил выявить оптимальное число кластеров и сбалансированное число объектов в кластерах. Полученные результаты понятны для интерпретации.